# Interpolation matrices for OpenDrift

***

In [35]:
import numpy as np
import xarray as xr
import netCDF4 as nc
import matplotlib.pyplot as plt
from scipy.interpolate import griddata

%matplotlib inline

***

## NEMO interpolation

Build target coordinates file

In [36]:
# Load NEMO mask and coordinates files
mask = xr.open_dataset('/data/bmoorema/MEOPAR/grid/mesh_mask201702.nc')
coords = xr.open_dataset('/data/bmoorema/MEOPAR/grid/coordinates_seagrid_SalishSea2.nc', decode_times=False)
fn_out = '/data/bmoorema/results/opendrift/OpenDrift_grid.nc'

# Define bounding box and resolution
lon_bounds, lat_bounds, res = [-126.1, -122.2], [47.9, 50.9], 0.005

# Construct target coordinates and mask arrays
lons_i, lats_i = [np.arange(*dim, res) for dim in [lon_bounds, lat_bounds]]
mask_i = griddata(
    tuple(coords[dim][0, ...].values.reshape(-1) for dim in ['glamt', 'gphit']),
    mask.tmask[0, 0, ...].values.reshape(-1), tuple(np.meshgrid(lons_i, lats_i)),
    method='nearest', fill_value=0,
)

# Write to netCDF out
with nc.Dataset(fn_out, 'w') as ds:
    for var, name in zip([lons_i, lats_i], ['longitude', 'latitude']):
        ds.createDimension(name, var.size)
        tempvar = ds.createVariable(name, float, name)
        tempvar[:] = var
    tempvar = ds.createVariable('tmask', float, ('latitude', 'longitude'))
    tempvar[:] = mask_i